<a href="https://colab.research.google.com/github/Rehabah/Real-Estate-Data/blob/main/Real_Estate_Data_Preprocessing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
conn = None
try:
    conn = psycopg2.connect("dbname = '' user = '' host = '' password = ''")
except psycopg2.DatabaseError:
    print ("I am unable to connect the database: " )
    sys.exit(1)

In [ ]:
Query_1=""" ;"""
Row1=pd.read_sql(Query_1, conn,index_col=None)

In [ ]:
len(Row1)

In [ ]:
Row1.columns

In [ ]:
Row1.head()

In [ ]:
Row1.isna().sum()

In [ ]:
pd.set_option('display.float_format', lambda x: '%0f' % x)
pd.set_option('display.max_colwidth', None)

In [ ]:
import datetime
Row1['collection_time1']=Row1['collection_time'].dt.date  

In [ ]:
data=Row1
len(data)

In [ ]:
data['property_location_district']=data['property_location_district'].str.replace('حي',"")
data['property_location_district']=data['property_location_district'].str.strip()

In [ ]:
data.loc[data['property_name'].str.contains('للإيجار'),'property_price_instalment']='إيجار'
data.loc[data['property_name'].str.contains('للبيع '),'property_price_instalment']='بيع'
data.loc[data['property_cat'].str.contains('RENT'),'property_price_instalment']='إيجار'
data.loc[data['property_cat'].str.contains('SALE'),'property_price_instalment']='بيع'

In [ ]:
#add condition null for property type
data.loc[data['property_cat'].str.contains('LAND '),'property_type']='أرض'
data.loc[data['property_cat'].str.contains('VILLA '),'property_type']='فيلا'
data.loc[data['property_cat'].str.contains('HOUSE '),'property_type']='بيت'
data.loc[data['property_cat'].str.contains('BUILDING '),'property_type']='عمارة'
data.loc[data['property_cat'].str.contains('APARTMENT '),'property_type']='شقة'
data.loc[data['property_cat'].str.contains('STORE '),'property_type']='محل'
data.loc[data['property_cat'].str.contains('FLOOR '),'property_type']='دور'

In [ ]:
len(data.loc[data['property_area'].str.contains( '</span>',na=False)])

In [ ]:
#meter price
data['property_price']=data['property_price'].str.replace(',',"")
data['property_price']=data['property_price'].str.strip() 
data['property_area']=data['property_area'].str.replace('م²',"")
data['property_area']=data['property_area'].str.strip()


In [ ]:
data['property_url1']=data['property_url'].str.replace('https://sa.aqar.fm/',"")
data['property_url1']=data['property_url1'].str.strip()

In [ ]:
data['property_area1']=data['property_area'].str.split().str[0]

In [ ]:
len(data.loc[data['property_area1']==''])

In [ ]:
data['street']=data['property_name'].str.split('شارع').str[-1]
data['street1']=data['street'].str.split('،').str[0]

In [ ]:
data.loc[~data['property_name'].str.contains("شارع ", case=False),'street1']='Unknown'

In [ ]:
data['property_time'].unique()

In [ ]:
data['property_time1']=''
data['property_time1']=data['collection_time1']

In [ ]:
data.loc[data['property_time'].str.contains('قبل يومين',na=False),'property_time']='قبل 2 يوم'
data.loc[data['property_time'].str.contains('قبل يوم',na=False),'property_time']='قبل 1 يوم'
data.loc[data['property_time'].str.contains('قبل شهرين',na=False),'property_time']='قبل 2 شهر'
data.loc[data['property_time'].str.contains('قبل شهر',na=False),'property_time']='قبل 1 شهر'
data.loc[data['property_time'].str.contains('قبل اسبوعين',na=False),'property_time']='قبل 2 اسبوع'
data.loc[data['property_time'].str.contains('قبل اسبوع',na=False),'property_time']='قبل 1 اسبوع'

In [ ]:
data['property_time']=data['property_time'].str.replace('قبل',"")
data['property_time']=data['property_time'].str.replace('أيام|ايام',"يوم")
data['property_time']=data['property_time'].str.replace('شهور','شهر')
data['property_time']=data['property_time'].str.replace('اسابيع','اسبوع')

In [ ]:
data.loc[data['property_time'].str.contains('يوم',na=False),'days']=data['property_time']
data.loc[data['property_time'].str.contains('شهر',na=False),'month']=data['property_time']
data.loc[data['property_time'].str.contains('اسبوع',na=False),'week']=data['property_time']

data['days1']=data['days'].str.extract('(\d+)')
data['month1']=data['month'].str.extract('(\d+)')
data['week1']=data['week'].str.extract('(\d+)')

data['days1']=data['days1'].fillna(0)
data['month1']=data['month1'].fillna(0)
data['week1']=data['week1'].fillna(0)

data["month1"] = data["month1"].astype(int)
data["week1"] = data["week1"].astype(int)

data['month1']=data['month1'].multiply(other =30)
data['week1']=data['week1'].multiply(other =7)

In [ ]:
data.loc[data['days1']==0,'days1']=data['month1']
data.loc[data['days1']==0,'days1']=data['week1']

In [ ]:
data['days1'].unique()

In [ ]:
data["days1"] = data["days1"].astype(int)
data["property_time1"]=data["collection_time1"]-pd.to_timedelta(data["days1"], unit='D')